Test and debug functions from spikingDataUtilities

In [1]:
import numpy as np
import spikingDataUtilities as su
%load_ext autoreload
%autoreload 2

In [4]:
# test restrict

# first column of samples is assumed to be time stamps at which all other columns are evaluated
samples = np.array([[1.1,5],[3,8],[4,-1.5],[7,2]])
# every row of intervals is an interval, sample time stamps must fall in at least one interval
intervals = np.array([[1,3.2],[6,8]])
su.restrict(samples,intervals,s_ind=True)

(array([[1.1, 5. ],
        [3. , 8. ],
        [7. , 2. ]]),
 array([ True,  True, False,  True]))

In [5]:
# test consolidateIntervals

intervals = np.array([[1,3.2],[6,8],[2,4],[9,11],[10,11],[11.1,12.1],[11.6,11.8]])
print(intervals[intervals[:, 0].argsort()])
su.consolidateIntervals(intervals)

[[ 1.   3.2]
 [ 2.   4. ]
 [ 6.   8. ]
 [ 9.  11. ]
 [10.  11. ]
 [11.1 12.1]
 [11.6 11.8]]


array([[ 1. ,  4. ],
       [ 6. ,  8. ],
       [ 9. , 11. ],
       [11.1, 12.1]])